# Scratch Notebook

In [1]:
import numpy as np
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.types import ArrayType, FloatType

In [2]:
data = [[float(x)] for x in range(10)]
df = spark.createDataFrame(
    data, schema=["a"]
)
df.show(); df.schema

+---+
|  a|
+---+
|0.0|
|1.0|
|2.0|
|3.0|
|4.0|
|5.0|
|6.0|
|7.0|
|8.0|
|9.0|
+---+



StructType([StructField('a', DoubleType(), True)])

In [3]:
def make_embedding_fn():
    def predict(a: np.ndarray) -> np.ndarray:
        print(f">>> {a=}")
        output = a * np.array([1.0, 2.0, 3.0])
        print(f">>> {output=}")
        return output
    return predict

embedding = predict_batch_udf(
    make_embedding_fn,
    return_type=ArrayType(FloatType()),
    batch_size=1
)

df.select(embedding("a")).show()

23/04/26 10:32:04 WARN TaskSetManager: Lost task 0.0 in stage 2.0 (TID 2) (192.168.86.223 executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/leey/devpub/spark/python/pyspark/worker.py", line 869, in main
    process()
  File "/home/leey/devpub/spark/python/pyspark/worker.py", line 861, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/leey/devpub/spark/python/pyspark/sql/pandas/serializers.py", line 354, in dump_stream
    return ArrowStreamSerializer.dump_stream(self, init_stream_yield_batches(), stream)
  File "/home/leey/devpub/spark/python/pyspark/sql/pandas/serializers.py", line 86, in dump_stream
    for batch in iterator:
  File "/home/leey/devpub/spark/python/pyspark/sql/pandas/serializers.py", line 347, in init_stream_yield_batches
    for series in iterator:
  File "/home/leey/devpub/spark/python/pyspark/worker.py", line 555, in func
    for result_batch, result_type in result_iter:
  File "/home/lee

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/home/leey/devpub/spark/python/pyspark/worker.py", line 869, in main
    process()
  File "/home/leey/devpub/spark/python/pyspark/worker.py", line 861, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/leey/devpub/spark/python/pyspark/sql/pandas/serializers.py", line 354, in dump_stream
    return ArrowStreamSerializer.dump_stream(self, init_stream_yield_batches(), stream)
  File "/home/leey/devpub/spark/python/pyspark/sql/pandas/serializers.py", line 86, in dump_stream
    for batch in iterator:
  File "/home/leey/devpub/spark/python/pyspark/sql/pandas/serializers.py", line 347, in init_stream_yield_batches
    for series in iterator:
  File "/home/leey/devpub/spark/python/pyspark/worker.py", line 555, in func
    for result_batch, result_type in result_iter:
  File "/home/leey/devpub/spark/python/pyspark/ml/functions.py"

## Create pandas/spark dataframes for testing

In [ ]:
import numpy as np
import pandas as pd

from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.functions import array, col, pandas_udf, spark_partition_id, struct
from pyspark.sql.types import *
from typing import Iterator, Union

In [ ]:
data = np.arange(0, 100000, dtype=np.float64).reshape(-1, 4)
pdf = pd.DataFrame(data, columns=['a','b','c','d'])

In [ ]:
pdf_scalar = pdf
df_scalar = spark.createDataFrame(pdf_scalar)

pdf_tensor = pd.DataFrame()
pdf_tensor['t1'] = pdf_scalar.values.tolist()
df_tensor1 = spark.createDataFrame(pdf_tensor)

pdf_tensor['t2'] = pdf_scalar.drop(columns='d').values.tolist()
df_tensor2 = spark.createDataFrame(pdf_tensor)

pdf_scalar_tensor = pdf
pdf_scalar_tensor['t1'] = pdf.values.tolist()

In [ ]:
pdf_scalar_tensor

In [ ]:
input_tensor_shapes = [None, None, None, None, [4]]

In [ ]:
inputs = [pdf_scalar_tensor[col] for col in pdf_scalar_tensor.columns]

In [ ]:
inputs = [np.vstack(v).reshape([-1] + input_tensor_shapes[i]) if input_tensor_shapes[i] else v for i, v in enumerate(inputs)]

In [ ]:
a,b,c,d,t1 = inputs

In [ ]:
np.sum(t1, axis=1) + a + b + c + d

In [ ]:
input_tensor_shapes = {4:[4]}

In [ ]:
input_shapes = [None] * 5
for index, shape in input_tensor_shapes.items():
    input_shapes[index] = shape

In [ ]:
input_shapes

### tests

In [ ]:
data = np.arange(0, 1000, dtype=np.float64).reshape(-1, 4)

# 4 scalar columns
pdf = pd.DataFrame(data, columns=["a", "b", "c", "d"])
pdf

In [ ]:
df = spark.createDataFrame(pdf)

In [ ]:
def multiples_with_array_fn():
    def predict(x, y):
        return {"x2": x * 2, "y3": y * 3}
    return predict

In [ ]:
multiples_w_array = predict_batch_udf(
    multiples_with_array_fn,
    return_type=StructType(
        [StructField("x2", DoubleType(), True), StructField("y3", ArrayType(DoubleType()), True)]
    ),
    input_tensor_shapes=[[], [3]],
    batch_size=5,
)
preds = df.withColumn("preds", multiples_w_array("a", array(["b", "c", "d"]))).select("a", "preds.*").toPandas()

### pandas dataframes

In [ ]:
pdf_scalar

In [ ]:
pdf_tensor

In [ ]:
foo = pdf_scalar['a']

In [ ]:
type(foo)

In [ ]:
bar = pdf_scalar['b']

In [ ]:
import numpy as np
import pandas as pd

data = np.arange(0, 4*100000, dtype=np.float64).reshape(-1, 4)
pdf = pd.DataFrame(data, columns=['a','b','c','d'])

In [ ]:
def batch(df, n=1):
    for _, batch in df.groupby(np.arange(len(df)) // n):
        yield batch

In [ ]:
def batch1(df, n=1):
    for i in range(0, len(df), n):
        yield df.iloc[i:i+n]

In [ ]:
def batch2(df, n=1):
    index = 0
    data_size = len(df)
    while index < data_size:
        yield df.iloc[index : index + n]
        index += n

In [ ]:
%%timeit -n 10 -r 10
for x in batch(pdf, 1000):
    x = x * 2

In [ ]:
%%timeit -n 10 -r 10
for x in batch1(pdf, 1000):
    x = x * 2

In [ ]:
%%timeit -n 10 -r 10
for x in batch2(pdf, 1000):
    x = x * 2

## Pydoc tests

In [ ]:
import numpy as np
import pandas as pd
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.types import ArrayType, FloatType, StructType, StructField
from typing import Mapping

data = np.arange(0, 1000, dtype=np.float64).reshape(-1, 4)
pdf = pd.DataFrame(data, columns=['a','b','c','d'])
pdf_tensor = pd.DataFrame()
pdf_tensor['t1'] = pdf.values.tolist()
pdf_tensor['t2'] = pdf.drop(columns='d').values.tolist()
df = spark.createDataFrame(pdf_tensor)
df.show(5)

In [ ]:
def multi_sum_fn():
    def predict(x1: np.ndarray, x2: np.ndarray) -> Mapping[str, np.dtype]:
        # x1.shape = [batch_size, 4]
        # x2.shape = [batch_size, 3]
        result = np.sum(x1, axis=1) + np.sum(x2, axis=1)
        # result.shape = [batch_size], result_type = FloatType()
        return result

    return predict

# multiple tensor columns with tensor_input_shapes => list of numpy arrays
sum_cols = predict_batch_udf(
    multi_sum_fn,
    return_type=FloatType(),
    batch_size=5,
    input_tensor_shapes=[[4], [3]],
)

In [ ]:
df.withColumn("sum", sum_cols("t1", "t2")).show(5)

In [ ]:
def make_multi_sum_fn():
    def predict_columnar(x1, x2):
        # x1.shape = [batch_size, 4]
        # x2.shape = [batch_size, 3]
        return {
            "sum1": np.sum(x1, axis=1),
            "sum2": np.sum(x2, axis=1)
        }  # return_type = StructType()

    return predict_columnar

sum_cols = predict_batch_udf(
    make_multi_sum_fn,
    return_type=StructType([
        StructField("sum1", FloatType(), True),
        StructField("sum2", FloatType(), True)
    ]),
    batch_size=5,
    input_tensor_shapes=[[4], [3]],
)

df.withColumn("sum", sum_cols("t1", "t2")).select("t1", "t2", "sum.*").show(5)

In [ ]:
def make_multi_sum_fn():
    def predict_row(x1: np.ndarray, x2: np.ndarray) -> list[Mapping[str, float]]:
        # x1.shape = [batch_size, 4]
        # x2.shape = [batch_size, 3]
        return [{'sum1': np.sum(x1[i]), 'sum2': np.sum(x2[i])} for i in range(len(x1))]
    return predict_row

In [ ]:
multi_sum_udf = predict_batch_udf(
    make_multi_sum_fn,
    return_type=StructType([
        StructField("sum1", FloatType(), True),
        StructField("sum2", FloatType(), True)
    ]),
    batch_size=5,
    input_tensor_shapes=[[4], [3]],
)

df.withColumn("sum", multi_sum_udf("t1", "t2")).select("t1", "t2", "sum.*").show(5)

In [ ]:
def make_multi_times_two_fn():
    def predict(x1: np.ndarray, x2: np.ndarray) -> Mapping[str, np.ndarray]:
        # x1.shape = [batch_size, 4]
        # x2.shape = [batch_size, 3]
        return {"t1x2": x1 * 2, "t2x2": x2 * 2}
    return predict

multi_times_two_udf = predict_batch_udf(
    make_multi_times_two_fn,
    return_type=StructType([
        StructField("t1x2", ArrayType(FloatType()), True),
        StructField("t2x2", ArrayType(FloatType()), True)
    ]),
    batch_size=5,
    input_tensor_shapes=[[4], [3]],
)

df.withColumn("x2", multi_times_two_udf("t1", "t2")).select("t1", "t2", "x2.*").show(5)

In [ ]:
import numpy as np
import pandas as pd
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.types import FloatType

df = spark.createDataFrame(pd.DataFrame(np.arange(100)))
df.show(5)

In [ ]:
def predict_batch_fn():
    def predict(inputs: np.ndarray) -> np.ndarray:
        # inputs.shape = [batch_size]
        # outputs.shape = [batch_size]  # return_type = FloatType()
        return inputs * 2

    return predict

times_two = predict_batch_udf(predict_batch_fn,
                              return_type=FloatType(),
                              batch_size=10)

In [ ]:
df.withColumn("x2", times_two("0")).show(5)

In [ ]:
def predict_batch_fn():
    def predict(inputs: np.ndarray) -> np.ndarray:
        # inputs.shape = [batch_size, 1]
        # outputs.shape = [batch_size]   # return_type = FloatType()
        return np.squeeze(inputs) * 3

    return predict

times_three = predict_batch_udf(predict_batch_fn,
                                return_type=FloatType(),
                                batch_size=10)

df = spark.createDataFrame(pd.DataFrame(np.arange(100)))
df.withColumn("x3", times_three("0")).show(5)

In [ ]:
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.functions import struct


In [ ]:
data = np.arange(0, 1000, dtype=np.float64).reshape(-1, 4)
pdf = pd.DataFrame(data, columns=['a','b','c','d'])
df = spark.createDataFrame(pdf)
df.show(5)

In [ ]:
import numpy as np
import pandas as pd
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.types import FloatType

df = spark.createDataFrame(pd.DataFrame(np.arange(100)))
df.show(5)

In [ ]:
def predict_batch_fn():
    def predict(inputs: np.ndarray) -> np.ndarray:
        # inputs.shape = [batch_size]
        # outputs.shape = [batch_size]  # return_type = FloatType()
        return inputs * 2

    return predict

times_two = predict_batch_udf(predict_batch_fn,
                              return_type=FloatType(),
                              batch_size=10)

In [ ]:
df.withColumn("x2", times_two("0")).show(5)

In [ ]:
def predict_batch_fn():
    def predict(inputs: np.ndarray) -> np.ndarray:
        # inputs.shape = [batch_size, 1]
        # outputs.shape = [batch_size]   # return_type = FloatType()
        return np.squeeze(inputs) * 3

    return predict

times_three = predict_batch_udf(predict_batch_fn,
                                return_type=FloatType(),
                                batch_size=10)

df = spark.createDataFrame(pd.DataFrame(np.arange(100)))
df.withColumn("x3", times_three("0")).show(5)

In [ ]:
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.functions import struct


In [ ]:
data = np.arange(0, 1000, dtype=np.float64).reshape(-1, 4)
pdf = pd.DataFrame(data, columns=['a','b','c','d'])
df = spark.createDataFrame(pdf)
df.show(5)

In [ ]:
import numpy as np
import pandas as pd
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.types import FloatType

df = spark.createDataFrame(pd.DataFrame(np.arange(100)))
df.show(5)

In [ ]:
def predict_batch_fn():
    def predict(inputs: np.ndarray) -> np.ndarray:
        # inputs.shape = [batch_size]
        # outputs.shape = [batch_size]  # return_type = FloatType()
        return inputs * 2

    return predict

times_two = predict_batch_udf(predict_batch_fn,
                              return_type=FloatType(),
                              batch_size=10)

In [ ]:
df.withColumn("x2", times_two("0")).show(5)

In [ ]:
def predict_batch_fn():
    def predict(inputs: np.ndarray) -> np.ndarray:
        # inputs.shape = [batch_size, 1]
        # outputs.shape = [batch_size]   # return_type = FloatType()
        return np.squeeze(inputs) * 3

    return predict

times_three = predict_batch_udf(predict_batch_fn,
                                return_type=FloatType(),
                                batch_size=10)

df = spark.createDataFrame(pd.DataFrame(np.arange(100)))
df.withColumn("x3", times_three("0")).show(5)

In [ ]:
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.functions import struct


In [ ]:
data = np.arange(0, 1000, dtype=np.float64).reshape(-1, 4)
pdf = pd.DataFrame(data, columns=['a','b','c','d'])
df = spark.createDataFrame(pdf)
df.show(5)

In [ ]:
import numpy as np
import pandas as pd
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.types import FloatType

df = spark.createDataFrame(pd.DataFrame(np.arange(100)))
df.show(5)

In [ ]:
def predict_batch_fn():
    def predict(inputs: np.ndarray) -> np.ndarray:
        # inputs.shape = [batch_size]
        # outputs.shape = [batch_size]  # return_type = FloatType()
        return inputs * 2

    return predict

times_two = predict_batch_udf(predict_batch_fn,
                              return_type=FloatType(),
                              batch_size=10)

In [ ]:
df.withColumn("x2", times_two("0")).show(5)

In [ ]:
def predict_batch_fn():
    def predict(inputs: np.ndarray) -> np.ndarray:
        # inputs.shape = [batch_size, 1]
        # outputs.shape = [batch_size]   # return_type = FloatType()
        return np.squeeze(inputs) * 3

    return predict

times_three = predict_batch_udf(predict_batch_fn,
                                return_type=FloatType(),
                                batch_size=10)

df = spark.createDataFrame(pd.DataFrame(np.arange(100)))
df.withColumn("x3", times_three("0")).show(5)

In [ ]:
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.functions import struct


In [ ]:
data = np.arange(0, 1000, dtype=np.float64).reshape(-1, 4)
pdf = pd.DataFrame(data, columns=['a','b','c','d'])
df = spark.createDataFrame(pdf)
df.show(5)

In [ ]:
np.sum(data, axis=1)

In [ ]:
    def predict_batch_fn():
        def predict(inputs: np.ndarray) -> np.ndarray:
            # inputs.shape = [batch_size, 4]
            # outputs.shape = [batch_size]   # return_type = FloatType()
            return np.sum(inputs, axis=1)

        return predict

    sum = predict_batch_udf(predict_batch_fn,
                            return_type=FloatType(),
                            batch_size=10,
                            input_tensor_shapes=[[4]])  

In [ ]:
    sum_rows = predict_batch_udf(predict_batch_fn,
                                 return_type=FloatType(),
                                 batch_size=10,
                                 input_tensor_shapes=[[4]])

    df.withColumn("sum", sum_rows(struct("a", "b", "c", "d"))).show(5)

In [ ]:
    def predict_batch_fn():
        def predict(x1: np.ndarray, x2: np.ndarray, x3: np.ndarray, x4: np.ndarray) -> np.ndarray:
            # xN.shape = [batch_size, 1]
            # outputs.shape = [batch_size, 1]   # return_type = ArrayType(FloatType())
            print("x1.shape: {}".format(x1.shape))
            return x1 + x2 + x3 + x4

        return predict
    
    sum_rows = predict_batch_udf(predict_batch_fn,
                                 return_type=FloatType(),
                                 batch_size=10)

    df.withColumn("sum", sum_rows(struct("a", "b", "c", "d"))).show(5)

### spark dataframes

In [ ]:
df_scalar.show()

In [ ]:
df_tensor1.show()

In [ ]:
df_tensor2.show()

### spark partitions

In [ ]:
from pyspark.sql.functions import spark_partition_id
df.withColumn("partition_id", spark_partition_id()).groupBy("partition_id").count().show()

## Test tensor columns

### pDF -> pDF | returnType=StructType | return pDF => OK

In [ ]:
df_tensor2.schema

In [ ]:
@pandas_udf(returnType=df_tensor2.schema)
def predict(inputs: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        print("===== batch.columns: {}".format(batch.columns))
        print("===== batch.dtypes:\n{}".format(batch.dtypes))
        yield batch

In [ ]:
columns = df_tensor2.columns
preds = df_tensor2.withColumn("preds", predict(struct(*columns))).toPandas()
preds

### pDF -> pDF | returnType=ArrayType(DoubleType()) | return pS => OK

In [ ]:
df_tensor2.schema

In [ ]:
@pandas_udf(returnType=ArrayType(DoubleType()))
def predict(inputs: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        print("===== batch.columns: {}".format(batch.columns))
        print("===== batch.dtypes:\n{}".format(batch.dtypes))
        yield batch['t1']

In [ ]:
columns = df_tensor2.columns
preds = df_tensor2.withColumn("preds", predict(struct(*columns))).toPandas()
preds

### pDF -> pS | returnType=StructType | return pDF => OK

In [ ]:
df_tensor2.schema

In [ ]:
@pandas_udf(returnType=df_tensor2.schema)
def predict(inputs: Iterator[pd.DataFrame]) -> Iterator[pd.Series]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        print("===== batch.columns: {}".format(batch.columns))
        print("===== batch.dtypes:\n{}".format(batch.dtypes))
        yield batch

In [ ]:
columns = df_tensor2.columns
preds = df_tensor2.withColumn("preds", predict(struct(*columns))).toPandas()
preds

### pDF -> pS | returnType=ArrayType(DoubleType()) | return pS => OK

In [ ]:
df_tensor2.schema

In [ ]:
@pandas_udf(returnType=ArrayType(DoubleType()))
def predict(inputs: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        print("===== batch.columns: {}".format(batch.columns))
        print("===== batch.dtypes:\n{}".format(batch.dtypes))
        yield batch['t1']

In [ ]:
columns = df_tensor2.columns
preds = df_tensor2.withColumn("preds", predict(struct(*columns))).toPandas()
preds

### Union[pDF, pS] -> Union[pDF, pS] | returnType=StructType() | return pS => FAIL

In [ ]:
df_tensor2.schema

In [ ]:
@pandas_udf(returnType=df_tensor2.schema)
# def predict(inputs: Iterator[Union[pd.DataFrame, pd.Series]]) -> Iterator[Union[pd.DataFrame, pd.Series]]:
# def predict(inputs: Iterator[pd.DataFrame]) -> Iterator[Union[pd.DataFrame, pd.Series]]:
# def predict(inputs: Iterator[pd.Series]) -> Iterator[Union[pd.DataFrame, pd.Series]]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        print("===== batch.columns: {}".format(batch.columns))
        print("===== batch.dtypes:\n{}".format(batch.dtypes))
        yield batch

## Test scalar columns

### pDF -> pDF | returnType=StructType | return pDF => OK

In [ ]:
df_scalar.schema

In [ ]:
@pandas_udf(returnType=df_scalar.schema)
def predict(inputs: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        print("===== batch.columns: {}".format(batch.columns))
        print("===== batch.dtypes:\n{}".format(batch.dtypes))
        yield batch

In [ ]:
columns = df_scalar.columns
preds = df_scalar.withColumn("preds", predict(struct(*columns))).toPandas()
preds

### pDF -> pDF | returnType=DoubleType() | return pS => OK

In [ ]:
df_scalar.schema

In [ ]:
@pandas_udf(returnType=DoubleType())
def predict(inputs: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        print("===== batch.columns: {}".format(batch.columns))
        print("===== batch.dtypes:\n{}".format(batch.dtypes))
        yield batch['a']

In [ ]:
columns = df_scalar.columns
preds = df_scalar.withColumn("preds", predict(struct(*columns))).toPandas()
preds

### pDF -> pS | returnType=StructType | return pDF => OK

In [ ]:
df_scalar.schema

In [ ]:
@pandas_udf(returnType=df_scalar.schema)
def predict(inputs: Iterator[pd.DataFrame]) -> Iterator[pd.Series]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        print("===== batch.columns: {}".format(batch.columns))
        print("===== batch.dtypes:\n{}".format(batch.dtypes))
        yield batch

In [ ]:
columns = df_scalar.columns
preds = df_scalar.withColumn("preds", predict(struct(*columns))).toPandas()
preds

### pDF -> pS | returnType=DoubleType() | return pS => OK

In [ ]:
df_scalar.schema

In [ ]:
@pandas_udf(returnType=DoubleType())
def predict(inputs: Iterator[pd.DataFrame]) -> Iterator[pd.Series]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        print("===== batch.columns: {}".format(batch.columns))
        print("===== batch.dtypes:\n{}".format(batch.dtypes))
        yield batch['a']

In [ ]:
columns = df_scalar.columns
preds = df_scalar.withColumn("preds", predict(struct(*columns))).toPandas()
preds

### Union[pDF, pS] -> Union[pDF, pS] | returnType=StructType) | return pS => FAIL

In [ ]:
@pandas_udf(returnType=df_scalar.schema)
def predict(inputs: Iterator[Union[pd.DataFrame, pd.Series]]) -> Iterator[Union[pd.DataFrame, pd.Series]]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        print("===== batch.columns: {}".format(batch.columns))
        print("===== batch.dtypes:\n{}".format(batch.dtypes))
        yield batch

In [ ]:
from inspect import signature
from pyspark.sql.pandas.typehints import infer_eval_type
from typing import get_type_hints, Any, Callable, Optional, Tuple

In [ ]:
def check_iterator_annotation(
    annotation: Any, parameter_check_func: Optional[Callable[[Any], bool]] = None
) -> bool:
    name = getattr(annotation, "_name", getattr(annotation, "__name__", None))
    return name == "Iterator" and (
        parameter_check_func is None or all(map(parameter_check_func, annotation.__args__))
    )

In [ ]:
def check_union_annotation(
    annotation: Any, parameter_check_func: Optional[Callable[[Any], bool]] = None
) -> bool:
    import typing

    # Note that we cannot rely on '__origin__' in other type hints as it has changed from version
    # to version. For example, it's abc.Iterator in Python 3.7 but typing.Iterator in Python 3.6.
    origin = getattr(annotation, "__origin__", None)
    return origin == typing.Union and (
        parameter_check_func is None or all(map(parameter_check_func, annotation.__args__))
    )

In [ ]:
def check_tuple_annotation(
    annotation: Any, parameter_check_func: Optional[Callable[[Any], bool]] = None
) -> bool:
    # Tuple has _name but other types have __name__
    # Check if the name is Tuple first. After that, check the generic types.
    name = getattr(annotation, "_name", getattr(annotation, "__name__", None))
    return name == "Tuple" and (
        parameter_check_func is None or all(map(parameter_check_func, annotation.__args__))
    )

In [ ]:
def predict(inputs: Iterator[Union[pd.Series, pd.DataFrame]]) -> Iterator[pd.DataFrame]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        print("===== batch.columns: {}".format(batch.columns))
        print("===== batch.dtypes:\n{}".format(batch.dtypes))
        yield batch

In [ ]:
sig = signature(predict)
sig

In [ ]:
type_hints = get_type_hints(predict)
type_hints

In [ ]:
annotations = {}
for param in sig.parameters.values():
    if param.annotation is not param.empty:
        annotations[param.name] = type_hints.get(param.name, param.annotation)
annotations

In [ ]:
parameters_sig = [
    annotations[parameter] for parameter in sig.parameters if parameter in annotations]
parameters_sig

In [ ]:
return_annotation = type_hints.get("return", sig.return_annotation)
return_annotation

In [ ]:
is_series_or_frame = all(
    a == pd.Series
    or a == pd.DataFrame  # Series
    or check_union_annotation(  # DataFrame  # Union[DataFrame, Series]
        a, parameter_check_func=lambda na: na == pd.Series or na == pd.DataFrame
    )
    for a in parameters_sig
) and (return_annotation == pd.Series or return_annotation == pd.DataFrame)
is_series_or_frame

In [ ]:
is_iterator_tuple_series_or_frame = (
    len(parameters_sig) == 1
    and check_iterator_annotation(  # Iterator
        parameters_sig[0],
        parameter_check_func=lambda a: check_tuple_annotation(  # Tuple
            a,
            parameter_check_func=lambda ta: (
                ta == Ellipsis
                or ta == pd.Series  # ...
                or ta == pd.DataFrame  # Series
                or check_union_annotation(  # DataFrame  # Union[DataFrame, Series]
                    ta, parameter_check_func=lambda na: (na == pd.Series or na == pd.DataFrame)
                )
            ),
        ),
    )
    and check_iterator_annotation(
        return_annotation, parameter_check_func=lambda a: a == pd.DataFrame or a == pd.Series
    )
)
is_iterator_tuple_series_or_frame

In [ ]:
is_iterator_series_or_frame = (
    len(parameters_sig) == 1
    and check_iterator_annotation(
        parameters_sig[0],
        parameter_check_func=lambda a: (
            a == pd.Series
            or a == pd.DataFrame  # Series
            or check_union_annotation(  # DataFrame  # Union[DataFrame, Series]
                a, parameter_check_func=lambda ua: ua == pd.Series or ua == pd.DataFrame
            )
        ),
    )
    and check_iterator_annotation(
        return_annotation, parameter_check_func=lambda a: a == pd.DataFrame or a == pd.Series
    )
)
is_iterator_series_or_frame

### pS -> pS | returnType=ArrayType(DoubleType()) | return pS => OK

In [ ]:
from pyspark.sql.functions import array

In [ ]:
df_scalar.schema

In [ ]:
@pandas_udf(returnType=ArrayType(DoubleType()))
def predict(inputs: Iterator[pd.Series]) -> Iterator[pd.Series]:
    for batch in inputs:
        print("===== batch: {}".format(type(batch)))
        print("===== len(batch: {}".format(len(batch)))
        # print("===== batch.columns: {}".format(batch.columns))
        # print("===== batch.dtypes:\n{}".format(batch.dtypes))
        print("===== batch[0]:\n{}".format(batch[0]))
        yield batch

In [ ]:
columns = df_scalar.columns
preds = df_scalar.withColumn("preds", predict(array(columns))).toPandas()
preds

## Test caching

In [ ]:
import numpy as np
import pandas as pd
import threading
import time

from pyspark.ml.functions import batch_infer_udf
from pyspark.sql.functions import struct, pandas_udf
from pyspark.sql.types import *
from typing import Iterator

In [ ]:
data = np.arange(0, 1000, dtype=np.float64).reshape(-1, 4)

# 4 scalar columns
pdf = pd.DataFrame(data, columns=["a", "b", "c", "d"])
df = spark.createDataFrame(pdf)

In [ ]:
def predict_batch_fn():
    # emulate loading a model, this should only be invoked once (per worker process)
    fake_output = np.random.random()

    def predict(inputs):
        return [fake_output for i in inputs]

    return predict

In [ ]:
identity = batch_infer_udf(predict_batch_fn, return_type=DoubleType())

In [ ]:
%%time
# results should be the same
df1 = df.withColumn("preds", identity(struct("a"))).toPandas()

In [ ]:
%%time
df2 = df.withColumn("preds", identity(struct("a"))).toPandas()

In [ ]:
%%time
df2 = df.withColumn("preds", identity(struct("a"))).toPandas()

In [ ]:
%%time
df2 = df.withColumn("preds", identity(struct("a"))).toPandas()

In [ ]:
%%time
df2 = df.withColumn("preds", identity(struct("a"))).toPandas()

## Test executors

In [ ]:
import numpy as np
import pandas as pd
import threading
import time

from pyspark.ml.functions import batch_infer_udf
from pyspark.sql.functions import struct, pandas_udf
from pyspark.sql.types import *
from typing import Iterator

In [ ]:
data = np.arange(0, 1000, dtype=np.float64).reshape(-1, 4)

# 4 scalar columns
pdf = pd.DataFrame(data, columns=["a", "b", "c", "d"])
df = spark.createDataFrame(pdf)

In [ ]:
def myfn(it):
    import tensorflow as tf
    print(">>>> {}".format(tf.__version__))
    print(tf.config.list_physical_devices('GPU'))
    tf.debugging.set_log_device_placement(True)

    # Create some tensors
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    c = tf.matmul(a, b)
    print(c)

In [ ]:
df.foreachPartition(myfn)

In [ ]:
@pandas_udf(returnType=FloatType())
def myudf(it: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
    for part in it:
        import tensorflow as tf
        print(tf.__version__)
        yield part

In [ ]:
foo = df.withColumn("preds", struct("a"))

In [ ]:
foo.show()

In [ ]:
print("==== df1:\n{}".format(df1))
print("==== df2:\n{}".format(df2))

In [ ]:
from pyspark.sql.functions import spark_partition_id
df.withColumn("partition_id", spark_partition_id()).withColumn("preds", identity(struct("a"))).groupBy("partition_id", "preds").count().show()

## Test zip

In [ ]:
import pandas as pd

In [ ]:
foo = pd.Series([0,1,2,3,4,5,6,7,8])
bar = pd.Series(['a','b','c','d','e','f','g','h','i'])

In [ ]:
test = (foo, bar)
test

In [ ]:
def batch(iterable, batch_size):
    acc = []
    for i, x in enumerate(zip(*iterable)):
        acc += x
        if i % batch_size == 0:
            yield acc
            acc = []
    yield acc

In [ ]:
import itertools
import more_itertools

def batch(iterable, batch_size):
    for x in more_itertools.chunked(zip(*iterable), batch_size):
        yield x

In [ ]:
for x in batch(test, 2):
    print(x)
    print("====")

In [ ]:
pdf = pd.concat(test, axis=1)

In [ ]:
pdf

In [ ]:
shape = [24, 24, 1]

In [ ]:
batch_shape = [-1] + shape
batch_shape